## Configure PySpark Setup

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"


import findspark
findspark.init()


import pyspark 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("App").getOrCreate()
spark

In [2]:
# check number of cores PySpark is using
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")

You are working with 1 core(s)


## Load Datasets


In [3]:
!cp /content/drive/MyDrive/Datasets.zip .
!unzip Datasets.zip

Archive:  Datasets.zip
  inflating: Datasets/.DS_Store      
  inflating: Datasets/fifa19.csv     
  inflating: Datasets/googleplaystore.csv  
  inflating: Datasets/nyc_air_bnb.csv  
  inflating: Datasets/people.json    
  inflating: Datasets/pga_tour_historical.csv  
  inflating: Datasets/rec-crime-pfa.csv  
  inflating: Datasets/Rep_vs_Dem_tweets.csv  
  inflating: Datasets/students.csv   
  inflating: Datasets/supermarket_sales.csv  
  inflating: Datasets/users1.parquet  
  inflating: Datasets/users2.parquet  
  inflating: Datasets/users3.parquet  
   creating: Datasets/uw-madison-courses/
  inflating: Datasets/uw-madison-courses/course_offerings.csv  
  inflating: Datasets/uw-madison-courses/courses.csv  
  inflating: Datasets/uw-madison-courses/database.sqlite3  
  inflating: Datasets/uw-madison-courses/grade_distributions.csv  
  inflating: Datasets/uw-madison-courses/instructors.csv  
  inflating: Datasets/uw-madison-courses/rooms.csv  
  inflating: Datasets/uw-madison-courses/s

# Load Libraries

In [18]:
from pyspark.sql.functions  import *
from  pyspark.sql.types  import *
import os

## Reading Data into PySpark

In [6]:
# create dummy dataset
valuesP = [('koala',1,'yes'),('caterpillar',2,'yes'),('deer',3,'yes'),('human',4,'yes')]
eats_plants = spark.createDataFrame(valuesP,['name','id','eats_plants'])

valuesM = [('shark',5,'yes'),('lion',6,'yes'),('tiger',7,'yes'),('human',4,'yes')]
eats_meat = spark.createDataFrame(valuesM,['name','id','eats_meat'])

print("Plant eaters (herbivores)")
print(eats_plants.show())

print("Meat eaters (carnivores)")
print(eats_meat.show())

Plant eaters (herbivores)
+-----------+---+-----------+
|       name| id|eats_plants|
+-----------+---+-----------+
|      koala|  1|        yes|
|caterpillar|  2|        yes|
|       deer|  3|        yes|
|      human|  4|        yes|
+-----------+---+-----------+

None
Meat eaters (carnivores)
+-----+---+---------+
| name| id|eats_meat|
+-----+---+---------+
|shark|  5|      yes|
| lion|  6|      yes|
|tiger|  7|      yes|
|human|  4|      yes|
+-----+---+---------+

None


In [9]:
# data appends

new_df = eats_plants

df_concat = eats_plants.union(new_df)


print(("eats_plants df Counts:", eats_plants.count(), len(eats_plants.columns)))
print(("df_concat Counts:", df_concat.count(), len(df_concat.columns)))
print(df_concat.show())


('eats_plants df Counts:', 4, 3)
('df_concat Counts:', 8, 3)
+-----------+---+-----------+
|       name| id|eats_plants|
+-----------+---+-----------+
|      koala|  1|        yes|
|caterpillar|  2|        yes|
|       deer|  3|        yes|
|      human|  4|        yes|
|      koala|  1|        yes|
|caterpillar|  2|        yes|
|       deer|  3|        yes|
|      human|  4|        yes|
+-----------+---+-----------+

None


In [11]:
# inner join
inner_join = eats_plants.join(eats_meat, on=["name","id"],how="inner")
inner_join.show()

+-----+---+-----------+---------+
| name| id|eats_plants|eats_meat|
+-----+---+-----------+---------+
|human|  4|        yes|      yes|
+-----+---+-----------+---------+



In [12]:
# left join
left_join = eats_plants.join(eats_meat, on=["name","id"], how='left') 
left_join.show()

+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|       deer|  3|        yes|     null|
|      human|  4|        yes|      yes|
|      koala|  1|        yes|     null|
|caterpillar|  2|        yes|     null|
+-----------+---+-----------+---------+



In [14]:
# conditional joins
conditional_join = eats_plants.join(eats_meat, on=["name","id"], how='left').filter(eats_meat['name'].isNull())
conditional_join.show()

+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|       deer|  3|        yes|     null|
|      koala|  1|        yes|     null|
|caterpillar|  2|        yes|     null|
+-----------+---+-----------+---------+



In [15]:
# right join
right_join = eats_plants.join(eats_meat,  on=["name","id"],how='right') 
right_join.show()

+-----+---+-----------+---------+
| name| id|eats_plants|eats_meat|
+-----+---+-----------+---------+
|shark|  5|       null|      yes|
|human|  4|        yes|      yes|
|tiger|  7|       null|      yes|
| lion|  6|       null|      yes|
+-----+---+-----------+---------+



In [16]:
full_outer_join = eats_plants.join(eats_meat, on=["name","id"],how='full')
full_outer_join.show()

+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|       deer|  3|        yes|     null|
|      shark|  5|       null|      yes|
|      human|  4|        yes|      yes|
|      tiger|  7|       null|      yes|
|       lion|  6|       null|      yes|
|      koala|  1|        yes|     null|
|caterpillar|  2|        yes|     null|
+-----------+---+-----------+---------+



In [19]:
path = "Datasets/uw-madison-courses/"

df_list = []
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filename_list = filename.split(".") #separate path from .csv
        df_name = filename_list[0]
        df = spark.read.csv(path+filename,inferSchema=True,header=True)
        df.name = df_name
        df_list.append(df_name)
        exec(df_name + ' = df')
        
print(df_list)

['courses', 'schedules', 'teachings', 'grade_distributions', 'subjects', 'subject_memberships', 'instructors', 'course_offerings', 'rooms', 'sections']


In [20]:
course_offerings.limit(4).toPandas()

,uuid,course_uuid,term_code,name
0,344b3ebe-da7e-314c-83ed-9425269695fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1092,Cooperative Education Prog
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1082,Cooperative Education Prog
2,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog
3,075da420-5f49-3dd0-93df-13e3c152e1b1,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1114,Cooperative Education Prog


In [21]:
instructors.show(4,False)

+-------+------------------+
|id     |name              |
+-------+------------------+
|761703 |JOHN ARCHAMBAULT  |
|3677061|STEPHANIE KANN    |
|788586 |KATHY PREM        |
|1600463|KRISTIN KLARKOWSKI|
+-------+------------------+
only showing top 4 rows



In [22]:
teachings.show(3)

+-------------+--------------------+
|instructor_id|        section_uuid|
+-------------+--------------------+
|       761703|45adf63c-48c9-365...|
|       761703|c6280e23-5e43-385...|
|       761703|9395dc21-15d1-3fa...|
+-------------+--------------------+
only showing top 3 rows



In [23]:
# joining all the datasets present in the list
step1 = teachings.join(instructors, teachings['instructor_id'] == instructors['id'], how='left').select(['instructor_id','name','section_uuid'])
step1.limit(4).toPandas()

,instructor_id,name,section_uuid
0,761703,JOHN ARCHAMBAULT,45adf63c-48c9-3659-8561-07556d2d4ddf
1,761703,JOHN ARCHAMBAULT,c6280e23-5e43-3859-893e-540d94993529
2,761703,JOHN ARCHAMBAULT,9395dc21-15d1-3fab-8d1f-6f3fe6114c48
3,3677061,STEPHANIE KANN,b99e440b-39db-350a-81eb-b6eb1bd8b0bc


In [24]:
step2 = step1.join(sections, step1['section_uuid'] == sections['uuid'], how='left').select(['name','course_offering_uuid'])
step2.limit(4).toPandas()

,name,course_offering_uuid
0,THOMAS JAHNS,f850ab24-740c-311a-a669-804a3fea7b0b
1,JEAN-FRANCOIS HOUDE,7e213b2b-c58b-3014-b3d1-01c0f7ed46ef
2,CHRISTOPHER R TABER,3beb7bd7-4877-3c63-8afc-62f8b74e72fc
3,MARISA S OTEGUI,db253216-2e66-3267-86b2-7b9f5fe07223


In [25]:
step3 = step2.withColumnRenamed('name', 'instructor').join(course_offerings, step2.course_offering_uuid == course_offerings.uuid, how='inner').select(['instructor','name','course_offering_uuid'])
step3.limit(4).toPandas()

,instructor,name,course_offering_uuid
0,THOMAS JAHNS,Master's Research or Thesis,f850ab24-740c-311a-a669-804a3fea7b0b
1,JEAN-FRANCOIS HOUDE,Wrkshp-Industrl Organizatn,7e213b2b-c58b-3014-b3d1-01c0f7ed46ef
2,CHRISTOPHER R TABER,Workshop - Public Economics,3beb7bd7-4877-3c63-8afc-62f8b74e72fc
3,MARISA S OTEGUI,Plant Cell Biology,db253216-2e66-3267-86b2-7b9f5fe07223
